In [35]:
import customtkinter
from customtkinter import CTkToplevel,CTkTextbox
import os
from PIL import ImageTk, Image

from pathlib import Path
from tkinter import filedialog
import customtkinter
from tkinter import filedialog
from pathlib import Path
import os 

import sys
import vtk
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import pydicom as dicom
from pydicom.data import get_testdata_file
from datetime import datetime


import pydicom
import numpy
import numpy as np
import cv2
import os
import math
import pylab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage
from skimage import morphology


import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import imageio

import pydicom as dicom
import numpy as np
import os

import customtkinter as ctk







import sys
import vtk
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import pydicom as dicom
from pydicom.data import get_testdata_file
from datetime import datetime


import pydicom
import numpy
import numpy as np
import cv2
import os
import math
import pylab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage
from skimage import morphology


import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import imageio

import pydicom as dicom
import numpy as np
import os

import matplotlib.pyplot as plt
from PIL import ImageTk, Image
import customtkinter as ctk

from skimage import exposure



In [98]:

app = customtkinter.CTk()
app.geometry("600x500")
app.title("main")






vol = None
dcm = None    
def browse_dicom_folder():
    global vol, dcm


    folder_path = filedialog.askdirectory()
    if folder_path:
        print("Selected DICOM folder:", folder_path)
        # Process the DICOM files in the folder as needed

        # Load the DICOM files from the specified folder
        dcm_files = os.listdir(folder_path)

        # Sort the DICOM files by instance number
        def get_instance_number(file_path_1):
            dcm = dicom.read_file(os.path.join(folder_path, file_path_1))
            return int(dcm.InstanceNumber)

        dcm_files.sort(key=lambda x: get_instance_number(x))

        # Load the pixel data from each DICOM file and stack into a 3D numpy array
        dcm = dicom.read_file(os.path.join(folder_path, dcm_files[0]))
        shape = (len(dcm_files), dcm.Rows, dcm.Columns)
        if 'NumberOfFrames' in dcm:
            shape = (dcm.NumberOfFrames, dcm.Rows, dcm.Columns)

        vol = np.zeros(shape, dtype=dcm.pixel_array.dtype)

        for i, file_path in enumerate(dcm_files):
            dcm = dicom.read_file(os.path.join(folder_path, file_path))
            if 'NumberOfFrames' in dcm:
                vol[i:i+dcm.NumberOfFrames,:,:] = dcm.pixel_array
            else:
                vol[i,:,:] = dcm.pixel_array

        # ---------------------------------------------------------------------------------------------------------------------------------------------------

        # The shape of the stacked images in each plane (Axial, Coronal, and Sagittal, respectively)
        n0, n1, n2 = vol.shape
        # Print the ouput
        #print("Number of Slices:\n\t", "Axial=", n0, "Slices\n\t","Coronal=", n1, "Slices\n\t","Sagittal=", n2, "Slices")

        # Extract the voxel spacing information
        spacing = [float(dcm.PixelSpacing[0]), float(dcm.PixelSpacing[1]), float(dcm.SliceThickness)]

        # Compute the sampling in each plane
        d0, d1, d2 = spacing[::-1] # Reverse order to match array indexing

        # Print the output
        # print("Sampling:\n\t", "Axial=", d2, "mm\n\t","Coronal=", d1, "mm\n\t","Sagittal=", d0, "mm")

        # The aspect ratio along the axial plane
        axial_asp = d1/d2
        # The aspect ratio along the sagittal plane
        sagittal_asp = d0/d1
        # The aspect ratio along the coronal plane
        coronal_asp = d0/d2
    else:
        print("file not found")








def view_dicom_image():

    global vol, dcm







    axial_image_index = None 
    def slider_event_axial(axial_slice):
        #original
        # axial_image_index = int(axial_slice)
        # axial_image = Image.fromarray(vol[axial_image_index,:,:])
        # axial_image = axial_image.resize((400, 400))
        # axial_photo = ImageTk.PhotoImage(axial_image)
        # image_label_axial.configure(image=axial_photo)
        # image_label_axial.image = axial_photo

        global axial_image_index 
        axial_image_index= int(axial_slice)
        axial_image_1 = vol[axial_image_index, :, :]  
        axial_image_1 = exposure.rescale_intensity(axial_image_1, in_range='uint16', out_range='uint8') 
        stretched_image_axial = exposure.rescale_intensity(axial_image_1, in_range=(0, 10)) 
        axial_image = Image.fromarray(stretched_image_axial)
        axial_image = axial_image.resize((400, 400))
        photo_axial = ImageTk.PhotoImage(axial_image)
        image_label_axial.configure(image=photo_axial)
        image_label_axial.image = photo_axial

    # bone segmentation function 
    def bone_segmentation_func_bone_only_axial():
        global axial_image_index
        bone_segmented_image_axial_slice = vol[axial_image_index,:,:]

        intercept = dcm.RescaleIntercept
        slope = dcm.RescaleSlope

        hu_image = bone_segmented_image_axial_slice * slope + intercept

        def window_image(original_image, window_center, window_width):

            img_min = window_center - window_width // 2

            img_max = window_center + window_width // 2

            window_image = original_image.copy()

            window_image[window_image < img_min] = img_min

            window_image[window_image > img_max] = img_max


            return window_image

        bone_image = window_image(hu_image, 400, 1000)

        
        global image_temp_gui_1
        image_temp_gui_1 = bone_image

        global image_gui_1
        image_gui_1 = Image.fromarray(image_temp_gui_1)
        image_gui_1 = image_gui_1.resize((400, 370))
        global photo1
        photo1 = ImageTk.PhotoImage(image_gui_1)



    coronal_image_index = None
    def slider_event_coronal(coronal_slice):
        #original
        # coronal_image_index = int(coronal_slice)
        # coronal_image = Image.fromarray(vol[:,coronal_image_index,:])
        # coronal_image = coronal_image.resize((400, 400))
        # coronal_photo = ImageTk.PhotoImage(coronal_image)
        # image_label_coronal.configure(image=coronal_photo)
        # image_label_coronal.image = coronal_photo
        global coronal_image_index
        coronal_image_index = int(coronal_slice)
        coronal_image_1 = vol[:,coronal_image_index,:]  
        coronal_image_1 = exposure.rescale_intensity(coronal_image_1, in_range='uint16', out_range='uint8') 
        stretched_image_coronal = exposure.rescale_intensity(coronal_image_1, in_range=(0, 10)) 
        coronal_image = Image.fromarray(stretched_image_coronal)
        coronal_image = coronal_image.resize((400, 400))
        photo_coronal = ImageTk.PhotoImage(coronal_image)
        image_label_coronal.configure(image=photo_coronal)
        image_label_coronal.image = photo_coronal

    # bone segmentation function 
    def bone_segmentation_func_bone_only_coronal():
        global coronal_image_index
        bone_segmented_image_coronal_slice = vol[:,coronal_image_index,:]
            
        intercept = dcm.RescaleIntercept
        slope = dcm.RescaleSlope
        hu_image_2 = bone_segmented_image_coronal_slice * slope + intercept
        
        def window_image_2(original_image, window_center, window_width):

            img_min = window_center - window_width // 2

            img_max = window_center + window_width // 2

            window_image = original_image.copy()

            window_image[window_image < img_min] = img_min

            window_image[window_image > img_max] = img_max


            return window_image
        bone_image_2 = window_image_2(hu_image_2, 400, 1000)

        global image_temp_gui_2
        image_temp_gui_2 = bone_image_2
        global image_gui_2
        image_gui_2 = Image.fromarray(image_temp_gui_2)
        image_gui_2 = image_gui_2.resize((400, 370))
        global photo2
        photo2 = ImageTk.PhotoImage(image_gui_2)



    sagittal_image_index =  None
    def slider_event_sagittal(sagittal_slice):
        #original
        # sagittal_image_index = int(sagittal_slice) 
        # sagittal_image = Image.fromarray(vol[:,:,sagittal_image_index])
        # sagittal_image = sagittal_image.resize((400, 400))
        # sagittal_photo = ImageTk.PhotoImage(sagittal_image)
        # image_label_sagittal.configure(image=sagittal_photo)
        # image_label_sagittal.image = sagittal_photo
        global sagittal_image_index
        sagittal_image_index = int(sagittal_slice) 
        sagittal_image_1 = vol[:,:,sagittal_image_index]  
        sagittal_image_1 = exposure.rescale_intensity(sagittal_image_1, in_range='uint16', out_range='uint8') 
        stretched_image_sagittal = exposure.rescale_intensity(sagittal_image_1, in_range=(0, 10)) 
        sagittal_image = Image.fromarray(stretched_image_sagittal)
        sagittal_image = sagittal_image.resize((400, 400))
        photo_sagittal = ImageTk.PhotoImage(sagittal_image)
        image_label_sagittal.configure(image=photo_sagittal)
        image_label_sagittal.image = photo_sagittal

    # bone segmentation function 
    def bone_segmentation_func_bone_only_sagittal():
        global sagittal_image_index
        bone_segmented_image_sagittal_slice = vol[:,:,sagittal_image_index]
        intercept = dcm.RescaleIntercept
        slope = dcm.RescaleSlope
        hu_image_3 = bone_segmented_image_sagittal_slice * slope + intercept

        def window_image_3(original_image, window_center, window_width):

            img_min = window_center - window_width // 2

            img_max = window_center + window_width // 2

            window_image = original_image.copy()

            window_image[window_image < img_min] = img_min

            window_image[window_image > img_max] = img_max


            return window_image
        bone_image_3 = window_image_3(hu_image_3, 400, 1000)
        
        global image_temp_gui_3
        image_temp_gui_3  = bone_image_3
        global image_gui_3
        image_gui_3 = Image.fromarray(image_temp_gui_3)
        image_gui_3 = image_gui_3.resize((400, 370))
        global photo3
        photo3 = ImageTk.PhotoImage(image_gui_3)







    
    def bone_segmentation_all():
        bone_segmentation_func_bone_only_axial()
        bone_segmentation_func_bone_only_coronal()
        bone_segmentation_func_bone_only_sagittal()








    def display_images():

        bone_segmentation_all()
        # Create CTkLabel widgets to display the images
        label1 = ctk.CTkLabel(toplevel_slicer)
        label1.configure(image = photo1)
        label1.grid(row=6, column=0, padx=10, pady=10)  # Position label at row 2, column 2
        label1.image = photo1  # Keep a reference to avoid garbage collection


        # Create CTkLabel widgets to display the images
        label2 = ctk.CTkLabel(toplevel_slicer)
        label2.configure(image = photo2)
        label2.grid(row=6, column=1, padx=10, pady=10)  # Position label at row 2, column 2
        label2.image = photo2  # Keep a reference to avoid garbage collection


        label3 = ctk.CTkLabel(toplevel_slicer)
        label3.configure(image = photo3)
        label3.grid(row=6, column=2, padx=10, pady=10)  # Position label at row 2, column 2
        label3.image = photo3  # Keep a reference to avoid garbage collection








    toplevel_slicer = CTkToplevel(app)  # master argument is optional  
    toplevel_slicer.title("DICOM Slicer")
    toplevel_slicer.geometry("1000x1600")

    # Create the sliders and position them using the grid layout
    slider_axial = ctk.CTkSlider(toplevel_slicer, from_=0, to= 365, command=slider_event_axial)
    slider_axial.grid(row=0, column=1, padx=10, pady=10)

    slider_coronal = ctk.CTkSlider(toplevel_slicer, from_=0, to= 511, command=slider_event_coronal)
    slider_coronal.grid(row=1, column=1, padx=10, pady=10)

    slider_sagittal = ctk.CTkSlider(toplevel_slicer, from_=0, to= 511, command=slider_event_sagittal)
    slider_sagittal.grid(row=2, column=1, padx=10, pady=10)

    #folder_path = r"C:\Users\LENOVO\Desktop\BMD 404\project\pic"
    #axial_image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
    #coronal_image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
    #sagittal_image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]

    # Create the text labels
    label_axial = ctk.CTkLabel(toplevel_slicer, text="Axial slice")
    label_axial.grid(row=0, column=0, padx=10, pady=10)

    label_coronal = ctk.CTkLabel(toplevel_slicer, text="Coronal slice")
    label_coronal.grid(row=1, column=0, padx=10, pady=10)

    label_sagittal = ctk.CTkLabel(toplevel_slicer, text="Sagittal slice")
    label_sagittal.grid(row=2, column=0, padx=10, pady=10)
    



    #axial
    #initial_axial_image = Image.fromarray(vol[180,:,:])
    #initial_axial_image = initial_axial_image.resize((400, 400))  # Resize the image to 200x200
    #initial_photo_axial = ImageTk.PhotoImage(initial_axial_image)
    initial_axial_image_1 = vol[180, :, :]  
    initial_axial_image_1 = exposure.rescale_intensity(initial_axial_image_1, in_range='uint16', out_range='uint8') 
    stretched_image = exposure.rescale_intensity(initial_axial_image_1, in_range=(0, 10)) 
    initial_axial_image = Image.fromarray(stretched_image)
    initial_axial_image = initial_axial_image.resize((400, 400))
    initial_photo_axial = ImageTk.PhotoImage(initial_axial_image)
    image_label_axial = ctk.CTkLabel(toplevel_slicer, image=initial_photo_axial)
    image_label_axial.grid(row=4, column=0, padx=10, pady=10)




    #coronal
    # initial_coronal_image = Image.fromarray(vol[:,255,:])
    # initial_coronal_image = initial_coronal_image.resize((400, 400))  # Resize the image to 200x200
    # initial_photo_coronal = ImageTk.PhotoImage(initial_coronal_image)
    initial_coronal_image_1 = vol[:,255,:]  
    initial_coronal_image_1 = exposure.rescale_intensity(initial_coronal_image_1, in_range='uint16', out_range='uint8') 
    stretched_image_coronal = exposure.rescale_intensity(initial_coronal_image_1, in_range=(0, 10)) 
    initial_coronal_image = Image.fromarray(stretched_image_coronal)
    initial_coronal_image = initial_coronal_image.resize((400, 400))
    initial_photo_coronal = ImageTk.PhotoImage(initial_coronal_image)
    image_label_coronal = ctk.CTkLabel(toplevel_slicer, image=initial_photo_coronal)
    image_label_coronal.grid(row=4, column=1, padx=10, pady=10)
    



    #sagittal
    # initial_sagittal_image = Image.fromarray(vol[:,:,100])
    # initial_sagittal_image = initial_sagittal_image.resize((400, 400))  # Resize the image to 200x200
    # initial_photo_sagittal = ImageTk.PhotoImage(initial_sagittal_image)
    initial_sagittal_image_1 = vol[:,:,255]  
    initial_sagittal_image_1 = exposure.rescale_intensity(initial_sagittal_image_1, in_range='uint16', out_range='uint8') 
    stretched_image_sagittal = exposure.rescale_intensity(initial_sagittal_image_1, in_range=(0, 10)) 
    initial_sagittal_image = Image.fromarray(stretched_image_sagittal)
    initial_sagittal_image = initial_sagittal_image.resize((400, 400))
    initial_photo_sagittal = ImageTk.PhotoImage(initial_sagittal_image)
    image_label_sagittal = ctk.CTkLabel(toplevel_slicer, image=initial_photo_sagittal)
    image_label_sagittal.grid(row=4, column=2, padx=10, pady=10)


    button_bone_segmentation = ctk.CTkButton(toplevel_slicer, text="Bone segmentation", command = display_images, hover = True)
    button_bone_segmentation.grid(row=5, column=0, padx=10, pady=10)


    toplevel_slicer.mainloop()








def browse_dicom_file(): # DONE
    global file_path

    filetypes = [("DICOM Files", "*.dcm")]
    initial_dir = os.getcwd()  # Set the initial directory to the current working directory
    file_path = filedialog.askopenfilename(filetypes=filetypes, initialdir=initial_dir)
    if file_path:
        print("Selected folder:", file_path)

def View_the_DICOM_file_information(): # DONE
    global ds,list_ct
    list_ct = dicom.read_file(file_path)
    ds = dicom.dcmread(file_path)
    print(list_ct)

    toplevel = CTkToplevel(app)  # master argument is optional  
    toplevel.title("DICOM file information")
    toplevel.geometry("600x500")

    #label = customtkinter.CTkLabel(toplevel, text=list_ct)
    #label.grid(row=1, column=0, padx=20, pady=20, sticky="w")
    
    text = ctk.CTkTextbox(toplevel,height=550, width=550)
    text.insert("1.0", list_ct)
    text.configure(state="disabled")
    text.pack(padx=20, pady=20)

    
    # Calculate required height and width based on text content
    text.update_idletasks()  # Ensure widget is updated
    height = text.get_size()[1] + 2  # Add a small padding
    width = text.get_size()[0]
    # Configure the widget with the calculated dimensions
    text.configure(height=height, width=width)


        
    toplevel.mainloop()

def About_us():
    toplevel_about_us = CTkToplevel(app)
    toplevel_about_us.title("DICOM file information")
    toplevel_about_us.geometry("600x500")


    # Create and position the text labels
    supervision_label = ctk.CTkLabel(toplevel_about_us, text="Under Supervision Of :", font=("Arial", 20, "bold","underline"))
    supervision_label.place(x=50, y=50)

    dr_label = ctk.CTkLabel(toplevel_about_us, text="Dr.Mohameed Ali Maher ", font=("Arial", 16,))
    dr_label.place(x=50, y=100)

    ta_label = ctk.CTkLabel(toplevel_about_us, text="TA Mennatallah Abdelsattar", font=("Arial", 16))
    ta_label.place(x=50, y=130)

    team_label = ctk.CTkLabel(toplevel_about_us, text="Our Team :", font=("Arial", 20, "bold","underline"))
    team_label.place(x=50, y=200)

    team_member_label1 = ctk.CTkLabel(toplevel_about_us, text="Ahmed Waleed Fahmy Ali -  19100838", font=("Arial", 16))
    team_member_label1.place(x=50, y=250)
    team_member_label2 = ctk.CTkLabel(toplevel_about_us, text="Abdeen Elsayed Ahmed - 19106051", font=("Arial", 16))
    team_member_label2.place(x=50, y=280)
    team_member_label3 = ctk.CTkLabel(toplevel_about_us, text="Mohammed Ahmed Abdelrahim - 19104825", font=("Arial", 16))
    team_member_label3.place(x=50, y=310)
    team_member_label4 = ctk.CTkLabel(toplevel_about_us, text="Mohamed Ahmed Mohamed - 19105014", font=("Arial", 16))
    team_member_label4.place(x=50, y=340)

    toplevel_about_us.mainloop()

def exit_fun(): # work in progress
    print("exit")




button_folder = customtkinter.CTkButton(app, text="Browse for DICOM Folder", command=browse_dicom_folder, hover=True)
button_folder.grid(row=0, column=0, padx=20, pady=20)

button_file = customtkinter.CTkButton(app, text="Browse for DICOM File", command=browse_dicom_file, hover=True)
button_file.grid(row=1, column=0, padx=20, pady=20)

button_view_dicom_info = customtkinter.CTkButton(app, text="View the DICOM file information", command=View_the_DICOM_file_information, hover=True)
button_view_dicom_info.grid(row=2, column=0, padx=20, pady=20)

view_dicom_image_button = customtkinter.CTkButton(app, text="View DICOM Image", command=view_dicom_image, hover=True)
view_dicom_image_button.grid(row=3, column=0, padx=20, pady=20)


about_us_button = customtkinter.CTkButton(app, text="About Us", command=About_us, hover=True)
about_us_button.grid(row=4, column=0, padx=20, pady=20)


exit_button = customtkinter.CTkButton(app, text="Exit", command=exit_fun, hover=True,fg_color="red", text_color="white")
exit_button.grid(row=5, column=0, padx=20, pady=20)


app.mainloop()



